### Tải thư viện Selenium
- Đối với máy tính cá nhân dùng lệnh: `pip install selenium webdriver_manager`
- Đối với colab, sử dụng cell bên dưới để tải selenium

In [ ]:
# %%shell
# # Ubuntu no longer distributes chromium-browser outside of snap
# #
# # Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# # Add debian buster
# cat > /etc/apt/sources.list.d/debian.list << "EOF"
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
# EOF

# # Add keys
# apt-key adv--keyserver keyserver.ubuntu.com--recv-keys DCC9EFBF77E11517
# apt-key adv--keyserver keyserver.ubuntu.com--recv-keys 648ACFD622F3D138
# apt-key adv--keyserver keyserver.ubuntu.com--recv-keys 112695A0E562B32A

# apt-key export 77E11517 | gpg--dearmour-o /usr/share/keyrings/debian-buster.gpg
# apt-key export 22F3D138 | gpg--dearmour-o /usr/share/keyrings/debian-buster-updates.gpg
# apt-key export E562B32A | gpg--dearmour-o /usr/share/keyrings/debian-security-buster.gpg

# # Prefer debian repo for chromium* packages only
# # Note the double-blank lines between entries
# cat > /etc/apt/preferences.d/chromium.pref << "EOF"
# Package: *
# Pin: release a=eoan
# Pin-Priority: 500


# Package: *
# Pin: origin "deb.debian.org"
# Pin-Priority: 300


# Package: chromium*
# Pin: origin "deb.debian.org"
# Pin-Priority: 700
# EOF

# # Install chromium and chromium-driver
# apt-get update
# apt-get install chromium chromium-driver

# # Install selenium
# pip install selenium

### Import các thư viện cần thiết

In [1]:
import pandas as pd
import re
import os
import requests
import time
import random

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### Khởi tạo Selenium driver    
Driver trong Selenium đóng vai trò như trình duyệt web, giúp thực hiện các thao tác như truy cập vào trang web dựa vào đường dẫn, thao tác chuyển trang,...

In [2]:
WEBDRIVER_DELAY_TIME_INT = 10
TIMEOUT_INT = 10
service = Service()
chrome_options = webdriver.EdgeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("window-size=1920x1080")
chrome_options.headless = True
driver = webdriver.Edge(service=service, options=chrome_options)
driver.implicitly_wait(TIMEOUT_INT)
wait = WebDriverWait(driver, WEBDRIVER_DELAY_TIME_INT)

### Trích xuất nội dung thơ
- Dữ liệu về các bài thơ sẽ được lấy từ trang web [thivien.net](https://www.thivien.net/)

In [3]:
def extract_poem_links(driver, url):
    driver.get(url)
    content_tags_xpath = '//*[@class="page-content container"]//div[@class="page-content-main"]//div[@class="list-item"]'
    content_tags = driver.find_elements(By.XPATH, content_tags_xpath)
    
    poem_links = []
    for tag in content_tags:
        try: 
            link_element = tag.find_element(By.XPATH, './/h4[@class="list-item-header"]/a')
            title = link_element.text
            link = link_element.get_attribute('href')
            poem_links.append({
                'title': title, 
                'link': link
            })
        except Exception as e: 
            print(f'Error extracting link: {e}')
            continue
    return poem_links

poem_links = []
for i in range(1, 11):
    links = extract_poem_links(driver, 
                           url=f'https://www.thivien.net/searchpoem.php?PoemType=16&ViewType=1&Country=2&Page={i}')
    poem_links.extend(links)
len(poem_links)

100

#### Tự code

In [4]:
def scrape_poems(driver, num_pages=10):
    datasets = [] 
    for page_idx in tqdm(range(1, num_pages + 1)):
        links = extract_poem_links(driver, 
                            url=f'https://www.thivien.net/searchpoem.php?PoemType=16&ViewType=1&Country=2&Page={page_idx}')
    
        for poem_link in links: 
            link = poem_link['link']
            try:
                driver.get(link)
                content = driver.find_element(By.XPATH, '//div[@class="poem-content"]/p')
                src_tag = driver.find_element(By.XPATH, '//div[@class="small"]')
                poem = {
                    'title': poem_link['title'],
                    'content': content.text, 
                    'source': src_tag.text, 
                    'link': link, 
                }
                datasets.append(poem)
            except Exception as e:
                print(f'Error processing {link}: {e}')
    return datasets  

In [5]:
datasets = scrape_poems(driver=driver)

  0%|          | 0/10 [00:00<?, ?it/s]

Error processing https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3%A2n/C%C3%A1i-l%C3%A0m-ta-h%E1%BA%A1nh-ph%C3%BAc/poem-t24-M5Dn2cxmEDVSrQhtCw: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class="poem-content"]/p"}
  (Session info: MicrosoftEdge=133.0.3065.92); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF706D8E735+13397]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF70701B234+2060404]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF706F72338+1368440]
	(No symbol) [0x00007FF706B632A9]
	(No symbol) [0x00007FF706B6349B]
	(No symbol) [0x00007FF706BA40E7]
	(No symbol) [0x00007FF706B8429F]
	(No symbol) [0x00007FF706B5964D]
	(No symbol) [0x00007FF706BA1C00]
	(No symbol) [0x00007FF706B83FC3]
	(No symbol) [0x00007FF706B58B96]
	(No symbol) [0x00007FF706B57E50]
	(No symbol) [0x0000

 10%|█         | 1/10 [00:36<05:27, 36.38s/it]

Error processing https://www.thivien.net/Th%C3%ADch-Thi%E1%BB%81n-T%C3%A2m/L%E1%BA%B7ng-ng%E1%BB%93i-ch%E1%BB%91n-t%C4%A9nh-l%C3%A2u/poem-6510NdSMoamB9S3QMp9AhA: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class="poem-content"]/p"}
  (Session info: MicrosoftEdge=133.0.3065.92); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF706D8E735+13397]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF70701B234+2060404]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF706F72338+1368440]
	(No symbol) [0x00007FF706B632A9]
	(No symbol) [0x00007FF706B6349B]
	(No symbol) [0x00007FF706BA40E7]
	(No symbol) [0x00007FF706B8429F]
	(No symbol) [0x00007FF706B5964D]
	(No symbol) [0x00007FF706BA1C00]
	(No symbol) [0x00007FF706B83FC3]
	(No symbol) [0x00007FF706B58B96]
	(No symbol) [0x00007FF706B57E50]

 20%|██        | 2/10 [00:57<03:41, 27.69s/it]

Error processing https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3%A2n/Ng%C6%B0%E1%BB%9Di-lu%C3%B4n-mi%E1%BB%87ng-k%C3%AAu-b%E1%BA%ADn/poem-uQlnz134n_GaUSZqo3rBvg: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class="poem-content"]/p"}
  (Session info: MicrosoftEdge=133.0.3065.92); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF706D8E735+13397]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF70701B234+2060404]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF706F72338+1368440]
	(No symbol) [0x00007FF706B632A9]
	(No symbol) [0x00007FF706B6349B]
	(No symbol) [0x00007FF706BA40E7]
	(No symbol) [0x00007FF706B8429F]
	(No symbol) [0x00007FF706B5964D]
	(No symbol) [0x00007FF706BA1C00]
	(No symbol) [0x00007FF706B83FC3]
	(No symbol) [0x00007FF706B58B96]
	(No symbol) [0x00007FF706B57E50]


100%|██████████| 10/10 [02:44<00:00, 16.41s/it]


In [6]:
# lưu thành file csv
df = pd.DataFrame(datasets)
df.to_csv('poem_dataset.csv', index=True)

In [7]:
df.head()

,title,content,source,link
0,“Bạn xấu như chiếc bóng”,Bạn xấu như chiếc bóng\nCứ bám riết theo anh\n...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
2,“Chơi thân không có nghĩa”,Chơi thân không có nghĩa\nKhông cãi nhau bao g...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
3,“Có thể buồn chút ít”,"Có thể buồn chút ít\nMột mình, không người yêu...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,“Có thể có ông bố”,Có thể có ông bố\nKhông yêu con; xưa nay\nĐã ô...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...


#### code AIO

In [11]:
def extract_poem_links(driver, page_idx):
    main_url = f"https://www.thivien.net/searchpoem.php?PoemType=16&ViewType=1&Country=2&Age[]=3&Page={page_idx}"
    driver.get(main_url)
    time.sleep(random.uniform(3, 5))

    content_tags_xpath = '//*[@class="page-content container"]//div[@class="page-content-main"]//div[@class="list-item"]'
    content_tags = driver.find_elements(By.XPATH, content_tags_xpath)
    poem_links = []
    for tag in content_tags:
        try:
            link_element = tag.find_element(By.XPATH, './/h4[@class="list-item-header"]/a')
            poem_title = link_element.text
            poem_url = link_element.get_attribute("href")
            poem_links.append({"title": poem_title, "url": poem_url})
        except Exception as e:
            print(f"Error extracting link: {e}")
            continue
    return poem_links

In [12]:
def clean_poem_html(html):
    html = re.sub(r"<img.*?>", "", html, flags=re.IGNORECASE)
    html = re.sub(r"<i>.*?</i>", "", html, flags=re.IGNORECASE | re.DOTALL)
    html = re.sub(r"<b>(.*?)</b>(?!\s*(?:<br\s*/?>\s*){2,})", r"\1", html, flags=re.IGNORECASE)
    html = re.sub(r"<br\s*/?>", "\n", html, flags=re.IGNORECASE)
    html = re.sub(r"</?p>", "", html, flags=re.IGNORECASE)

    return html.strip()

def process_poem_content(html, poem_src, poem_url, default_title=""):
    cleaned = clean_poem_html(html)

    pattern = re.compile(r"<b>(.*?)</b>\s*\n{2,}", flags=re.IGNORECASE)
    matches = list(pattern.finditer(cleaned))

    poems = []
    if matches:
        for i, match in enumerate(matches):
            title = match.group(1).strip()
            start = match.end()
            end = matches[i+1].start() if i + 1 < len(matches) else len(cleaned)
            content = cleaned[start:end].strip("\n")
            poems.append({
                "title": title,
                "content": content,
                "source": poem_src,
                "url": poem_url
            })
    else:
        poems.append({
            "title": default_title,
            "content": cleaned,
            "source": poem_src,
            "url": poem_url
        })
    return poems

In [13]:
def scrape_poem(driver, poem_url):
    driver.get(poem_url)
    time.sleep(random.uniform(3, 5))

    poem_content_tag = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, "div.poem-content"))
    )

    html_content = poem_content_tag.get_attribute("innerHTML")

    try:
        poem_src_tag = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//div[@class="small"]'))
        )
        poem_src = poem_src_tag.text
    except Exception:
        poem_src = ""
    return process_poem_content(html_content, poem_src, poem_url)

def scrape_poems(driver, num_pages=10):
    datasets = []
    for page_idx in tqdm(range(1, num_pages + 1)):
        poem_links = extract_poem_links(driver, page_idx)
        for poem in poem_links:
            poem_url = poem["url"]
            try:
                poems = scrape_poem(driver, poem_url)
                datasets.extend(poems)
            except Exception as e:
                print(f"Error processing {poem_url}: {e}")
                continue
    return datasets

In [14]:
datasets = scrape_poems(driver, num_pages=10)
driver.quit()

  0%|          | 0/10 [00:00<?, ?it/s]

 40%|████      | 4/10 [04:03<06:02, 60.39s/it]

Error processing https://www.thivien.net/Tu%E1%BB%87-S%E1%BB%B9/18/poem-vvXMqDr3PYay-7iOBtJn6A: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF706D8E735+13397]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF70701B234+2060404]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF706F72338+1368440]
	(No symbol) [0x00007FF706B632A9]
	(No symbol) [0x00007FF706B6349B]
	(No symbol) [0x00007FF706BA40E7]
	(No symbol) [0x00007FF706B8429F]
	(No symbol) [0x00007FF706B5964D]
	(No symbol) [0x00007FF706BA1C00]
	(No symbol) [0x00007FF706B83FC3]
	(No symbol) [0x00007FF706B58B96]
	(No symbol) [0x00007FF706B57E50]
	(No symbol) [0x00007FF706B589C3]
	(No symbol) [0x00007FF706BE0214]
	(No symbol) [0x00007FF706C706CF]
	(No symbol) [0x00007FF706BE3A53]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF7070D463D+279981]
	simdutf::get_active_implementation [0x00007FF706D178C1+409425]
	simdutf::get_active_implementation [0x00007FF706D12BA4+389684]
	simdutf::g

 90%|█████████ | 9/10 [12:16<01:31, 91.70s/it]

Error processing https://www.thivien.net/L%C3%AA-Th%E1%BB%8B-Kim/Ai-n%C3%ADu-m%C3%B9a-xu%C3%A2n-%C4%91%E1%BA%A5y/poem-mHKUylp0Ay4_lEHEyPiDHw: HTTPConnectionPool(host='localhost', port=55828): Read timed out. (read timeout=120)
Error processing https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%BA%A3ng-H%C3%A0n/Ai-v%E1%BB%81-khuya/poem-Ryze7jz3QInxVqCI5EnYBg: HTTPConnectionPool(host='localhost', port=55828): Read timed out. (read timeout=120)
Error processing https://www.thivien.net/Nguy%E1%BB%85n-Thu%E1%BB%B5-Kha/%C3%81m-%E1%BA%A3nh/poem-cbnm6wwjWHryVa3JGYypuA: HTTPConnectionPool(host='localhost', port=55828): Read timed out. (read timeout=120)
Error processing https://www.thivien.net/T%C3%B4-H%C3%A0/%C3%81m-%E1%BA%A3nh/poem-KW4MtfU0KfNqvVvvLIFv0A: HTTPConnectionPool(host='localhost', port=55828): Read timed out. (read timeout=120)
Error processing https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%BA%A3ng-H%C3%A0n/%C3%81m-%E1%BA%A3nh-s%C3%B4ng-x%C6%B0a/poem-J56x5YUFt2msQ9DpMTHETQ: HTTPConnectio

100%|██████████| 10/10 [25:43<00:00, 154.36s/it]


In [15]:
# Lưu thành file csv
df2 = pd.DataFrame(datasets)
df2.to_csv('poem_dataset2.csv', index=True)

In [16]:
df2.head()

,title,content,source,url
0,,Bạn xấu như chiếc bóng\nCứ bám riết theo anh\n...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
3,,Chơi thân không có nghĩa\nKhông cãi nhau bao g...,[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,,"Có thể buồn chút ít\nMột mình, không người yêu...",[Thông tin 1 nguồn tham khảo đã được ẩn],https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
